In [ ]:
# Get the stock price data

import yfinance as yf
import riskfolio as rp
import warnings
warnings.filterwarnings("ignore")
# Riskfolio-Lib prints out some warnings that are safe to ignore. So ignore them.

# After the imports, create a list of tickers you want to use. Use any tickers you want. Since you can download data for more than one stock at a time, it’s one line of code to get all the data.

# portfolio tickers
assets = ["JCI", "TGT", "CMCSA", "CPB", "MO", "APA", "MMC", "JPM",
          "ZION", "PSA", "BAX", "BMY", "LUV", "PCAR", "TXT", "TMO",
          "DE", "MSFT", "HPQ", "SEE", "VZ", "CNP", "NI", "T", "BA"]

# sort tickers
assets.sort()

#download data
data = yf.download(assets, start="2016-01-01", end="2019-12-30")

# compute non-compounding, daily returns
returns = data['Adj Close'].pct_change().dropna()

In [ ]:
# Step 2: Create a portfolio with equal risk weights

# First, setup a portfolio with equal risk weights. This means Riskfolio-Lib will find the weights that cause the risk contribution of each stock to be equal.

port = rp.Portfolio(returns=returns)

port.assets_stats(method_mu='hist', method_cov='hist', d=0.94)

w_rp = port.rp_optimization(
    model="Classic",  # use historical
    rm="MV",  # use mean-variance optimization
    hist=True,  # use historical scenarios
    rf=0,  # set risk free rate to 0
    b=None  # don't use constraints
)
# First, build the portfolio object with the stock returns. Then estimate the expected returns and covariance based on historic data. Finally, use the classical mean-variance optimization to find the risk parity weights.

# Riskfolio-Lib makes it easy to visualize the weights.

ax = rp.plot_pie(w=w_rp)

In [ ]:
# show the risk contribution for each asset is equal
ax = rp.plot_risk_con(
    w_rp,
    cov=port.cov,
    returns=port.returns,
    rm="MV",
    rf=0,
)

In [ ]:
# A criticism of risk parity is that without leverage, returns lag. So add a constraint to weight the stocks in a way to reach a minimum portfolio return. This adds weight to higher-risk stocks to push the portfolio returns higher.

port.lowerret = 0.0008

# estimate the optimal portfolio with risk parity with the constraint
w_rp_c = port.rp_optimization(
    model="Classic",  # use historical
    rm="MV",  # use mean-variance optimization
    hist=True,  # use historical scenarios
    rf=0,  # set risk free rate to 0
    b=None  # don't use constraints
)
# Add a constraint for the minimum level of expected returns for the entire portfolio.

# Again, plotting is easy.

ax = rp.plot_pie(w=w_rp_c)

In [ ]:
# Plot the risk contributions.

ax = rp.plot_risk_con(
    w_rp_c,
    cov=port.cov,
    returns=port.returns,
    rm="MV",
    rf=0,
)